In [ ]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 50 kB/s 
     |████████████████████████████████| 199 kB 62.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=56d12a7ee1c1932418ebfeb2dcdaa9b8db709d7c171da9e87895f955101bd2d2
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


Questions

1.   Find the total cost, total revenue, total profit on the basis of each region
2. Find the Item List on the basis of each country
3. Find the total number of items sold in each country
4. Find the top five famous item lists on the basis of each region.(Consider units sold while doing this.)
5. Find all the regions and their famous sales channels.
6. Find  the list of countries and items and their respective units.
7. In 2013, identify the regions which sold maximum and minimum units of item type Meat.
8. List all the items whose unit cost is less than 500
9. Find the total cost, revenue and profit of each year.






In [ ]:
from pyspark.sql import SparkSession , functions as F, Window as W
spark = SparkSession.builder.appName('sales').getOrCreate()

In [ ]:
from pyspark.sql.types import StringType, FloatType, IntegerType

In [ ]:
df_sales = spark.read.format('json').load('sample_data/sales_records.json')
df_sales.printSchema()
df_sales.show(10)

root
 |-- Country: string (nullable = true)
 |-- Item Type: string (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Order Priority: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Sales Channel: string (nullable = true)
 |-- Ship Date: string (nullable = true)
 |-- Total Cost: string (nullable = true)
 |-- Total Profit: string (nullable = true)
 |-- Total Revenue: string (nullable = true)
 |-- Unit Cost: string (nullable = true)
 |-- Unit Price: string (nullable = true)
 |-- Units Sold: string (nullable = true)
 |-- _corrupt_record: string (nullable = true)

+--------------------+-------------+----------+---------+--------------+--------------------+-------------+----------+----------+------------+-------------+---------+----------+----------+---------------+
|             Country|    Item Type|Order Date| Order ID|Order Priority|              Region|Sales Channel| Ship Date|Total Cost|Total Profit|Total Revenu

In [ ]:
#typecasting arithmetic attributes
df_sales = df_sales\
            .withColumn('Total Cost', df_sales['Total Cost'].cast(FloatType()))\
            .withColumn('Total Profit', df_sales['Total Profit'].cast(FloatType()))\
            .withColumn('Total Revenue', df_sales['Total Revenue'].cast(FloatType()))\
            .withColumn('Unit Cost', df_sales['Unit Cost'].cast(FloatType()))\
            .withColumn('Unit Price', df_sales['Unit Price'].cast(FloatType()))\
            .withColumn('Units Sold', df_sales['Units Sold'].cast(IntegerType()))
df_sales.printSchema()
df_sales.show(10)

root
 |-- Country: string (nullable = true)
 |-- Item Type: string (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Order Priority: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Sales Channel: string (nullable = true)
 |-- Ship Date: string (nullable = true)
 |-- Total Cost: float (nullable = true)
 |-- Total Profit: float (nullable = true)
 |-- Total Revenue: float (nullable = true)
 |-- Unit Cost: float (nullable = true)
 |-- Unit Price: float (nullable = true)
 |-- Units Sold: integer (nullable = true)
 |-- _corrupt_record: string (nullable = true)

+--------------------+-------------+----------+---------+--------------+--------------------+-------------+----------+----------+------------+-------------+---------+----------+----------+---------------+
|             Country|    Item Type|Order Date| Order ID|Order Priority|              Region|Sales Channel| Ship Date|Total Cost|Total Profit|Total Revenue|Un

In [97]:
df_sales.coalesce(1).write.csv('Output/sales/sales_data',header=True,mode='overwrite')

##1. Find the total cost, total revenue, total profit on the basis of each region


In [ ]:
# to droping null in region attribute
df_sales = df_sales.na.drop(subset=['Region'])

In [102]:
question1=df_sales.groupBy('Region').agg(F.sum('Total Cost'), F.sum('Total Revenue'), F.sum('Total Profit'))
question1.show()

+--------------------+--------------------+--------------------+--------------------+
|              Region|     sum(Total Cost)|  sum(Total Revenue)|   sum(Total Profit)|
+--------------------+--------------------+--------------------+--------------------+
|Middle East and N...|1.194003115362056...|1.691834583280536...| 4.978314680986322E9|
|Australia and Oce...| 7.526098663258995E9|1.070152222371284...|3.1754235603909473E9|
|              Europe|2.415937816242763E10|3.423977049206286...|1.008039233312204...|
|  Sub-Saharan Africa|2.465031758112777...|3.495487197307492E10|1.030455438786905...|
|Central America a...|1.026651963972375...|1.455373016365411...|4.2872105216859627E9|
|       North America| 2.064450719133194E9|2.9370023334700775E9| 8.725516170647297E8|
|                Asia|1.358588970253493...|1.929340122315577...|5.7075115161706505E9|
+--------------------+--------------------+--------------------+--------------------+



In [107]:
question1.toPandas().to_csv('Output/sales/question1',index= False)

## 2. Find the Item List on the basis of each country

In [103]:
question2=df_sales.withColumn('Itemlist', F.collect_set('Item Type').over(W.partitionBy('Country'))).select('Country', 'Itemlist').distinct()
question2.show()

+--------------------+--------------------+
|             Country|            Itemlist|
+--------------------+--------------------+
|         Afghanistan|[Beverages, Perso...|
|             Albania|[Beverages, Perso...|
|             Algeria|[Beverages, Perso...|
|             Andorra|[Beverages, Perso...|
|              Angola|[Beverages, Perso...|
|Antigua and Barbuda |[Beverages, Perso...|
|             Armenia|[Beverages, Perso...|
|           Australia|[Beverages, Perso...|
|             Austria|[Beverages, Perso...|
|          Azerbaijan|[Beverages, Perso...|
|             Bahrain|[Beverages, Perso...|
|          Bangladesh|[Beverages, Perso...|
|            Barbados|[Beverages, Perso...|
|             Belarus|[Beverages, Perso...|
|             Belgium|[Beverages, Perso...|
|              Belize|[Beverages, Perso...|
|               Benin|[Beverages, Perso...|
|              Bhutan|[Beverages, Perso...|
|Bosnia and Herzeg...|[Beverages, Perso...|
|            Botswana|[Beverages

In [117]:
question2.toPandas().to_csv('Output/sales/question2',index= False)

##3. Find the total number of items sold in each country


In [108]:
question3=df_sales.groupBy('Country').agg(F.sum('Units Sold').alias('Total number of items sold'))
question3.show()

+-----------+--------------------------+
|    Country|Total number of items sold|
+-----------+--------------------------+
|       Chad|                   2660461|
|     Russia|                   2579558|
|      Yemen|                   2966519|
|    Senegal|                   2716010|
|     Sweden|                   2698756|
|   Kiribati|                   2555774|
|    Eritrea|                   2552497|
|Philippines|                   2610149|
|   Djibouti|                   2699545|
|      Tonga|                   2565238|
|  Singapore|                   2693579|
|   Malaysia|                   2587267|
|       Fiji|                   2613373|
|     Turkey|                   2732629|
|     Malawi|                   2645975|
|       Iraq|                   2765491|
|    Germany|                   2502470|
|    Comoros|                   2556790|
|   Cambodia|                   2946963|
|Afghanistan|                   2805640|
+-----------+--------------------------+
only showing top

In [109]:
question3.toPandas().to_csv('Output/sales/question3',index= False)

##4. Find the top five famous items list on the basis of each region.(Consider units sold while doing this.)


In [ ]:
df_regionsales = df_sales.groupBy('Region', 'Item Type').agg(F.sum('Units Sold').alias('Total Units Sold')).orderBy('Region', 'Total Units Sold')
df_regionsales_rank = df_regionsales.withColumn('rank', F.rank().over(W.partitionBy('Region').orderBy(F.col('Total Units Sold').desc()))).where('rank <= 5')
df_regionsales_rank.show()            
                            

+--------------------+---------------+----------------+----+
|              Region|      Item Type|Total Units Sold|rank|
+--------------------+---------------+----------------+----+
|                Asia|         Cereal|         6397658|   1|
|                Asia|         Snacks|         6330265|   2|
|                Asia|Office Supplies|         6187421|   3|
|                Asia|     Vegetables|         6121703|   4|
|                Asia|        Clothes|         6117405|   5|
|Australia and Oce...|  Personal Care|         3616014|   1|
|Australia and Oce...|     Vegetables|         3569149|   2|
|Australia and Oce...|         Cereal|         3535286|   3|
|Australia and Oce...|      Beverages|         3436051|   4|
|Australia and Oce...|        Clothes|         3393563|   5|
|Central America a...|      Cosmetics|         4814795|   1|
|Central America a...|        Clothes|         4675554|   2|
|Central America a...|           Meat|         4675081|   3|
|Central America a...|Of

In [110]:
df_regionsales_rank.toPandas().to_csv('Output/sales/question4',index= False)

##5. Find all the regions and their famous sales channels

In [111]:
question5=df_sales.withColumn('Sales Channels', F.collect_set('Sales Channel').over(W.partitionBy('Region'))).select('Region', 'Sales Channels').distinct()
question5.show()


+--------------------+-----------------+
|              Region|   Sales Channels|
+--------------------+-----------------+
|                Asia|[Online, Offline]|
|Australia and Oce...|[Online, Offline]|
|Central America a...|[Online, Offline]|
|              Europe|[Online, Offline]|
|Middle East and N...|[Online, Offline]|
|       North America|[Online, Offline]|
|  Sub-Saharan Africa|[Online, Offline]|
+--------------------+-----------------+



In [112]:
question5.toPandas().to_csv('Output/sales/question5',index= False)

##6. Find the list of countries and items and their respective units.


In [113]:
question6=df_sales.groupBy('Country', 'Item Type').agg(F.sum('Units Sold').alias('Units Sold')).orderBy('Country')
question6.show()

+-----------+---------------+----------+
|    Country|      Item Type|Units Sold|
+-----------+---------------+----------+
|Afghanistan|     Vegetables|    219937|
|Afghanistan|      Baby Food|    232084|
|Afghanistan|Office Supplies|    166911|
|Afghanistan|           Meat|    273402|
|Afghanistan|        Clothes|    220429|
|Afghanistan|  Personal Care|    255956|
|Afghanistan|         Cereal|    256936|
|Afghanistan|      Beverages|    206154|
|Afghanistan|      Household|    261953|
|Afghanistan|         Fruits|    257336|
|Afghanistan|         Snacks|    237350|
|Afghanistan|      Cosmetics|    217192|
|    Albania|         Cereal|    215238|
|    Albania|           Meat|    266123|
|    Albania|        Clothes|    250884|
|    Albania|         Snacks|    210384|
|    Albania|Office Supplies|    236822|
|    Albania|      Baby Food|    191480|
|    Albania|         Fruits|    252283|
|    Albania|  Personal Care|    222874|
+-----------+---------------+----------+
only showing top

In [114]:
question6.toPandas().to_csv('Output/sales/question6',index= False)

##7. In 2013, identify the regions which sold maximum and minimum units of item type Meat.


In [115]:
df_sales_2013=df_sales.withColumn('year', F.substring('Order Date', -4, 4)).where(F.col('year') == '2013')

question7=df_sales_2013.where(df_sales['Item Type'] == 'Meat').groupBy('Region', 'Item Type').agg(F.sum('Units Sold').alias('Units Sold')).orderBy('Units Sold')
question7.show()


+--------------------+---------+----------+
|              Region|Item Type|Units Sold|
+--------------------+---------+----------+
|       North America|     Meat|    106193|
|Australia and Oce...|     Meat|    449346|
|Central America a...|     Meat|    615706|
|Middle East and N...|     Meat|    745940|
|                Asia|     Meat|    956367|
|              Europe|     Meat|   1468932|
|  Sub-Saharan Africa|     Meat|   1491277|
+--------------------+---------+----------+



In [118]:
question7.toPandas().to_csv('Output/sales/question7',index= False)

##8. List all the items whose unit cost is less than 500


In [119]:
question8=df_sales.filter(df_sales['Unit Cost'] < 500).select('Item Type', 'Unit Cost').distinct()
question8.show()

+-------------+---------+
|    Item Type|Unit Cost|
+-------------+---------+
|       Cereal|   117.11|
|         Meat|   364.69|
|    Baby Food|   159.42|
|       Fruits|     6.92|
|   Vegetables|    90.93|
|    Beverages|    31.79|
|Personal Care|    56.67|
|      Clothes|    35.84|
|    Cosmetics|   263.33|
|       Snacks|    97.44|
+-------------+---------+



In [120]:
question8.toPandas().to_csv('Output/sales/question8',index= False)

## 9. Find the total cost, revenue and profit of each year.


In [121]:
new_df_sales = df_sales.withColumn('Year', F.substring('Order Date', -4, 4))\

question9=new_df_sales.groupBy('Year').agg(F.sum('Total Cost'), F.sum('Total Revenue'), F.sum('Total Profit')).orderBy('Year')
question9.show()

+----+--------------------+--------------------+--------------------+
|Year|     sum(Total Cost)|  sum(Total Revenue)|   sum(Total Profit)|
+----+--------------------+--------------------+--------------------+
|2010|1.233298120617372...|1.752972613683793...|  5.19674493173579E9|
|2011|1.233635124975034...|1.751684120852689...| 5.180489954286942E9|
|2012|1.245034203973387...|1.762118501057573...| 5.170842975163134E9|
|2013|1.254475780787666...|1.780262842300091...| 5.257870612787804E9|
|2014| 1.26472629863539E10|1.786939230240254...| 5.222129311173204E9|
|2015|1.256502197052444...|1.779198425534374...| 5.226962291379013E9|
|2016|1.229711736956142...|1.746406771565606...| 5.166950335527011E9|
|2017| 7.018850991852474E9|1.000281918959213...|2.9839682052368097E9|
+----+--------------------+--------------------+--------------------+



In [122]:
question9.toPandas().to_csv('Output/sales/question9',index= False)